# Pandas Tutorial


<em> <p style = "text-indent:10em"> adapted from .....  Brandon Rhodes, PyCon15 </p> </em>

[GitHub](https://github.com/brandon-rhodes/pycon-pandas-tutorial.git)


[Data](pages.stern.nyu.edu/~dbackus/csv/)


[Pandas API](http://pandas.pydata.org/pandas-docs/stable/api.html)

In [ ]:
%matplotlib inline

import random
import numpy as np
import pandas as pd

import seaborn

# pd display option
pd.options.display.max_columns = 6
pd.options.display.max_rows = 25

from IPython.core.display import HTML

css = open('style-table.css').read() #+ open('custom.css').read()
HTML('<style>{}</style>'.format(css))


## Data format
[series](#Series)

Series are list/1D array like objects

In [ ]:
s = pd.Series([1,2,3,4,5])
s

[DataFrame](#DataFrame)

DataFrames are dict/multi dimensional array like objects 

In [ ]:
titles = pd.DataFrame.from_csv(
    'pycon-pandas-tutorial/titles.csv', 
    index_col = None, 
    encoding = 'utf-8')
titles

You may see some glitch in the notebook whenever a cell with existing ouput is ran. When a cell is executed (Shift/Ctrl+Enter), Ipython notebook discards the existing output. DOM of the browser kicks in clearing the bottom screen and scrolls up, since there is no need for that extra blank space. Ipython then produces a new output which falls below the view scope.

In [ ]:
titles.loc[214579:214582, 'year']

In [ ]:
titles.iloc[214579:214582] #location by index

## head and tail 

slices and creates a new DataFrame

In [ ]:
h = titles.head(20)
h

# Series

## Filtering by Rows & Columns

In [ ]:
h['title']
h.title

In [ ]:
h[['title', 'year']] 
# Note the [] inside [];  returns only the chosen colums

In [ ]:
h.ix[0:4]

## Operations

Any mathermatical operations can be performed on a **Series**

In [ ]:
h['year'] // 10 + 1000

In [ ]:
# Comparison operator returns boolean (True|False)
after2000 = h['year'] > 2000
after2000

In [ ]:
h[h['year'] < 1985]

#The output of the Series (boolean) is fed into the DataFrame. 
#Only the True values of DF is returned

In [ ]:
#Is any|all element True over requested axis

after2000.any(), after2000.all()

### Gotcha1

`h[h['year'] < 1985 and h['year'] >= 1990]` will throw an Exception error: The truth value of a Series is ambiguous.

`and` in this scenario proceeds by two steps: 
    
    STEP1: Compare the left argument 
    `h['year'] < 1985` to be True or False.
    
    STEP2: Move on and compare right argument 
    `h['year'] >= 1990`.

`and` can compare only single value not a Series/list of values.

#### Solution

**`&`** is a bitwise opertor of `and`. It can compare a list of bits against other.

The only problem is operator precedence (PEMDAS :)). For an input 

<pre><code>`h[h['year'] < 1985 & h['year'] >= 1990]` </code></pre>


**`&`** first compares '1990' and 'h.year' first and then compares the outer. So use `()` to order the operator precedence.

In [ ]:
h[(h['year'] < 1990) & (h['year'] <= 1971)]
h[(h['year'] < 1990) | (h['year'] <= 1971)]

### Gotcha2

**evaluating != assigning**
<pre><code>
x = 10; h = titles.head(10)
x + 10; h[h.year > 1980]
x     ; h  
</pre> </code>

x is still 10 ! 20 ; h is still `titles.head(10)`

### Gotcha3

**variables are just 'labels'**

    x = pd.Series([1,2,3,4])
    y = x
    y[0] = 2000
    x

In [ ]:
x = pd.Series([1,2,3,4])
y = x
y[0] = 2000
x

## copy

In [ ]:
x = pd.Series([1,2,3,4])
y = x
x = y.copy()
y[0] = 2000
x

## Ordering / Sorting

`order` a Series, `sort` a DataFrame 

In [ ]:
h['year'].order()

see [Gotcha4](### Gotcha4)

## apply,  map, drop

`apply` works on a row / column basis of a DataFrame, 

`applymap` works element-wise on a DataFrame, 

`map` works element-wise on a Series.

In [ ]:
h.ix[5:10] #h.head(5:10) !!!!

In [ ]:
def f(x):
    if x >= 1950:
        return str("19th Century")
    else:
        return str("20th Century")

In [ ]:
df = h.copy()
df['Century'] = h["year"].apply(f)
df.ix[5:10]

In [ ]:
dec = lambda x: x // 10*10
df['decade'] = df['year'].map(dec)

df = df.drop('Century', axis=1) #axis = 0 is rowwise operation.
df.ix[5:10]

## Freq and Plot

In [ ]:
#value_counts of a Series returns the frequency.
df.decade.value_counts().head(10)

### Gotcha4

The resulting Series will be sorted  by *values*. 


If we have a Series with *index* and *values*, we can use `.plot()` to vizualise.

In [ ]:
titles.year.value_counts().sort_index().plot()

In [ ]:
cast = pd.DataFrame.from_csv(
    'pycon-pandas-tutorial/cast.csv',
    index_col = None,
    encoding = 'utf-8')
cast.head(10)

In [ ]:
c = cast
c = c[c.character == 'Bosco']
c.plot(x='year', y='n', kind='scatter')

# DataFrame

In [ ]:
cast = pd.DataFrame.from_csv(
    'pycon-pandas-tutorial/cast.csv',
    index_col = None,
    encoding = 'utf-8')
cast.head(10)

In [ ]:
len(cast), list(cast)

In [ ]:
c = cast.head(10)
c = c[c.n.isnull()] #.notnull()
c[c.name == "Too $hort"]

## Index / Indices

Organise and fast access of data 

In [ ]:
cast.head()

In [ ]:
%%time 
#timeit runs the cell four times, Avoid it
cast[cast.title == 'Sleuth']

In [ ]:
c = cast.set_index(['title'])
c.head()

## DataFrame acts like a dict

After changing the usefull column as index, the DataFrame behaves like a 'dict'.

`.loc['key']` acts as dict look up. similar to `x['key']` in dict


In [ ]:
%time
c.loc['Sleuth']

### Trick
An index helps if it is ordered. 'divide and conquer' alogorithm can operate fast on a sorted index

In [ ]:
c = cast.set_index(['title']).sort_index()
c.head()

In [ ]:
%%time
c.loc['Sleuth']

**A DataFrame can have multiple index !!!**

In [ ]:
c = cast.set_index(['title', 'year']).sort_index()
c.head()

In [ ]:
%%time
#c.loc['Sleuth'].loc[2007]
c.loc[('Sleuth', 2007)]

## Reseting index

In [ ]:
c.loc[('Sleuth', 2007)].reset_index('year')

In [ ]:
c.loc[('Sleuth', 2007)].reset_index(['year', 'title']).set_index('type')

## GroupBy

In [ ]:
c = cast
c = c[c.name == 'George Clooney']
c[c['year'] == 1999]

In [ ]:
c.groupby(['year']).size()

In [ ]:
c.groupby(['title', 'year', 'character']).size() 

#try .mean() . IS this automatically checking the dtype and computing mean?

### Gotcha

`c.groupby(['title', 'year', 'character']).size()` works, but `c.groupby(c['title', 'year', 'character'])` doesn't.
`c.groupby(['title']).size()` works and `c.groupby(c['title'])` also works.


In [ ]:
c[['title', 'year', 'character']].head()

In [ ]:
c.groupby(['year']).size().plot(kind='bar', ylim = 0)

In [ ]:
#c.groupby(['year'])['n'].size()
c.groupby(['year'])['n'].mean()

### Trick

Numpy array like operations on a DataFrame or Series

    h['year'] + 100

    h['year'] > 1997

Trunctating division operator `//`. The first slice (`/`) does the division and the second slice (`/`) does truncates the fraction after the number.

    h['year'] // 10

To get decade use: 
    
    h['year'] // 10 * 10

In [ ]:
c.groupby(c['year'] // 10*10).size()

#Note the c['year]. First create decade and then groupby.
#c.groupby(c['year'] > 2000).size()
#c[c['year'] > 2000].groupby(by='year').size()
c.groupby([c['year'] // 10*10, 'year']).size()

In [ ]:
c = cast.set_index(['title']).sort_index()
c.groupby([c['year'] // 10*10, 'type']).size()

## Stack and Unstack (Pivot tables!)


    vertical (row) = unstack (up)
    horizont (col) = stack (side/down)

Stack brings everything towards LEFT

Unstack moves everything UP

In [ ]:
cast.unstack().unstack()

In [ ]:
c = cast
c = c[(c['character'] == 'Kermit the Frog') | (c['character'] == 'Oscar the Grouch')]
g = c.groupby(['character', c.year// 10*10]).size()
g

Consider the above group by output as as a **tuple** with (character, year). 
Now, unstack which ever element `(0|1)` or `('character'|'year')` as the column names.

In [ ]:
g.unstack(1)

#### fillna

In [ ]:
u = g.unstack(0).fillna(0)
u

In [ ]:
diff = u['Kermit the Frog'] - u['Oscar the Grouch']
diff

In [ ]:
u.stack('character')
# can only stack by the element in top-left corner 

### Trick

Double unstack a series by adding a dummy column

In [ ]:
g.unstack().unstack()
g

In [ ]:
un = g.unstack(1)
un['dummy'] = 1
un = un.set_index('dummy', append=True)
un

In [ ]:
un.unstack(0)

# Extras

In [ ]:
h

In [ ]:
h_copy = h.copy()
h_copy["rand"] = 1
h_copy

In [ ]:
h_copy["rand_date"] = pd.DataFrame(np.random.randint(1, high=30,size=len(h_copy["rand"])) , dtype=str)
h_copy["rand_month"] = pd.DataFrame(np.random.randint(1, high=12,size=len(h_copy["rand"])), dtype=str)
h_copy["rand_year"] = pd.DataFrame(h_copy["year"], dtype=str)

h_copy['rand_day'] = h_copy["rand_date"]+"-"+h_copy["rand_month"]+"-"+h_copy["rand_year"]

h_copy = h_copy.drop(['rand','rand_date','rand_month','rand_year'], axis=1)
h_copy

## Series methods

#### Srting

A namespace `str` is attahced for string manipulations `Series.str.startswith('YYY')`

In [ ]:
h1 = h_copy
h1[h1.title.str.startswith('U')]

In [ ]:
h1['newcol'] = 0
h1.loc[h1.title.str.startswith('A') | h1.title.str.startswith('U'), 'newcol'] = 1
g = h1.groupby(by = ['newcol'])
g.groups

In [ ]:
g = h1.groupby(by = h1.title.str.get(0))
g.groups

In [ ]:
h1.loc[g.groups['A'],:]
# h1.loc[h1.title.str.startswith('A')]

#### Date
`Series.dt.year`

In [ ]:
h1['rand_day'] = pd.to_datetime(h1['rand_day'])
h1.rand_day.dt.dayofyear

#### Pivot